In [10]:
import pandas as pd
import numpy as np

In [11]:
pop_JL_2018=pd.read_csv("/home/jian/Docs/EASI/2018-06-25/ZIP4_18_DATA_A2_CSV/Total_HH_POP_by_Zip_JL_20180723.csv",dtype=str)
pop_JL_2018['ZIP_CODE']=pop_JL_2018['ZIP_CODE'].astype(int)

udsmapping=pd.read_excel("/home/jian/Docs/Household_and_Population/2016/zip_to_zcta_2017_From udsmapper.xlsx",dtype=str)
udsmapping=udsmapping[~udsmapping['STATE'].isin(["AS","FM","GU","MH","MP","PW","PR","VI","AE","AA","AP"])]
print(len(udsmapping['STATE'].unique()))

udsmapping=udsmapping[['ZIP_CODE','ZCTA']]
udsmapping['ZIP_CODE']=udsmapping['ZIP_CODE'].astype(int)
pop_JL_2018_ZCTA=pd.merge(pop_JL_2018,udsmapping,on="ZIP_CODE",how="left")
pop_JL_2018_ZCTA['HH18']=pop_JL_2018_ZCTA['HH18'].astype(float)
pop_JL_2018_ZCTA['POP18']=pop_JL_2018_ZCTA['POP18'].astype(float)

51


In [12]:
pop_JL_2018_ZCTA.head(2)

,ZIP_CODE,HH18,POP18,ZCTA
0,501,0.0,0.0,11742
1,1001,6853.6,16240.7,01001


In [13]:
udsmapping.head(2)

,ZIP_CODE,ZCTA
0,501,11742
1,544,11742


In [14]:
udsmapping_Not_in_EASI=udsmapping[~udsmapping['ZCTA'].isin(pop_JL_2018_ZCTA['ZCTA'])] 
# Note1: for a group of zips of a ZCTA in udsmapping, if partial of the zips are included in spencer's file, the rest zips are
# not included in the final dataset, because there's no weight to apply to such zips

# Note2: for a group of zips of a ZCTA in udsmapping, if all of the zips are not included in spencer's file, all of these zips
# are given a -1 value of POP15 and HH15 for next step to evenly allocated the difference about 2%
udsmapping_Not_in_EASI['POP18']=-1
udsmapping_Not_in_EASI['HH18']=-1
udsmapping_Not_in_EASI=udsmapping_Not_in_EASI[pop_JL_2018_ZCTA.columns]
pop_JL_2018_ZCTA=pop_JL_2018_ZCTA.append(udsmapping_Not_in_EASI)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
result=pd.DataFrame()
ilit_time=0
for ZCTA, df in pop_JL_2018_ZCTA.groupby('ZCTA'):
    total_df_pop=df['POP18'].sum()
    total_df_hh=df['HH18'].sum()
    if total_df_pop!=0:
        df['Pop_Ratio']=df['POP18'].apply(lambda x: np.round(x/total_df_pop,6))    
    elif total_df_pop==0:
        df['Pop_Ratio']=1/len(df)
    else:
        df['Pop_Ratio']='nan'
    
    if total_df_hh!=0:
        df['HH_Ratio']=df['HH18'].apply(lambda x: np.round(x/total_df_hh,6))
    elif total_df_hh==0:
        df['HH_Ratio']=1/len(df)
    else:
        df['HH_Ratio']='nan'
    
    result=result.append(df)

    # 20+ minutes of running
    ilit_time=ilit_time+1
    if ilit_time % 1000==1:
        print(ilit_time)
     
result=result.sort_values('ZIP_CODE')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#inde

1
1001
2001
3001
4001
5001
6001
7001
8001
9001
10001
11001
12001
13001
14001
15001
16001
17001
18001
19001
20001
21001
22001
23001
24001
25001
26001
27001
28001
29001
30001
31001
32001


In [16]:
result['ZCTA']=result['ZCTA'].astype(int)
result_NoAjust=result[['ZIP_CODE','ZCTA']]
result_Pop=result[['ZIP_CODE','ZCTA','Pop_Ratio']]
result_HH=result[['ZIP_CODE','ZCTA','HH_Ratio']]

In [17]:
Data_1_DemoPopu=pd.read_excel("/home/jian/Docs/Household_and_Population/2016/ACS_16_5YR_DP05_with_ann (Demographic Population).xlsx",skiprows=1)
Data_1_DemoPopu=Data_1_DemoPopu.rename(columns={"Id2":"ZCTA"})

Data_1_DemoPopu_Col=Data_1_DemoPopu.columns.tolist()
Data_1_DemoPopu_Col=[x for x in Data_1_DemoPopu_Col if "Estimate" in x]

Data_1_DemoPopu_Col_NoAdjust=['Estimate; SEX AND AGE - Median age (years)']
Data_1_DemoPopu_Col_HH=[x for x in Data_1_DemoPopu_Col if "housing" in x]
no_pop_ajusted_num_col=Data_1_DemoPopu_Col_NoAdjust+Data_1_DemoPopu_Col_HH
Data_1_DemoPopu_Col_Pop=[x for x in Data_1_DemoPopu_Col if x not in no_pop_ajusted_num_col]

Data_1_NoAdjust=Data_1_DemoPopu[['ZCTA']+Data_1_DemoPopu_Col_NoAdjust]
Data_1_NoAdjust=pd.merge(result_NoAjust,Data_1_NoAdjust,on=['ZCTA'],how='left')

Data_1_PopAdjust=Data_1_DemoPopu[['ZCTA']+Data_1_DemoPopu_Col_Pop]
Data_1_PopAdjust=pd.merge(result_Pop,Data_1_PopAdjust,on=['ZCTA'],how='left')
for col in Data_1_DemoPopu_Col_Pop:
    Data_1_PopAdjust[col]=Data_1_PopAdjust[col]*Data_1_PopAdjust['Pop_Ratio']

Data_1_HHAdjust=Data_1_DemoPopu[['ZCTA']+Data_1_DemoPopu_Col_HH]
Data_1_HHAdjust=pd.merge(result_HH,Data_1_HHAdjust,on=['ZCTA'],how='left')
for col in Data_1_DemoPopu_Col_HH:
    Data_1_HHAdjust[col]=Data_1_HHAdjust[col]*Data_1_HHAdjust['HH_Ratio']
Data_1=pd.merge(Data_1_PopAdjust,Data_1_HHAdjust,on=["ZIP_CODE","ZCTA"],how="outer")
Data_1=pd.merge(Data_1,Data_1_NoAdjust,on=["ZIP_CODE","ZCTA"],how="outer")
del Data_1['Pop_Ratio']
del Data_1['HH_Ratio']

Data_1_DemoPopu_No_Zip=Data_1_DemoPopu[~Data_1_DemoPopu['ZCTA'].isin(Data_1['ZCTA'])]
Data_1_DemoPopu_No_Zip['ZIP_CODE']=np.nan
Data_1_DemoPopu_No_Zip=Data_1_DemoPopu_No_Zip[Data_1.columns.tolist()]
Data_1=Data_1.append(Data_1_DemoPopu_No_Zip)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [18]:
Data_2_HouseComp=pd.read_csv("/home/jian/Docs/Household_and_Population/2016/ACS_16_5YR_S1101_with_ann (Household Composition).csv",skiprows=1)

Data_2_HouseComp=Data_2_HouseComp.rename(columns={"Id2":"ZCTA"})

Data_2_HouseComp_Col=Data_2_HouseComp.columns.tolist()
Data_2_HouseComp_Col=[x for x in Data_2_HouseComp_Col if "Estimate" in x]

Data_2_NoAdjust_Col=[x for x in Data_2_HouseComp_Col if "size" in x]
Data_2_HHAdjust_Col=[x for x in Data_2_HouseComp_Col if "Total households" in x]
Data_2_HHAdjust_Col=['Total; Estimate; Total households',
                     'Married-couple family household; Estimate; Total households',
                     'Male householder, no wife present, family household; Estimate; Total households',
                     'Female householder, no husband present, family household; Estimate; Total households',
                     'Nonfamily household; Estimate; Total households',
                    ]

Data_2_NoAdjust=Data_2_HouseComp[['ZCTA']+Data_2_NoAdjust_Col]
Data_2_NoAdjust=pd.merge(result_NoAjust,Data_2_NoAdjust,on=['ZCTA'],how='left')

Data_2_HHAdjust=Data_2_HouseComp[['ZCTA']+Data_2_HHAdjust_Col]
Data_2_HHAdjust=pd.merge(result_HH,Data_2_HHAdjust,on=['ZCTA'],how='left')
for col in Data_2_HHAdjust_Col:
    Data_2_HHAdjust[col]=Data_2_HHAdjust[col]*Data_2_HHAdjust['HH_Ratio']
Data_2=pd.merge(Data_2_NoAdjust,Data_2_HHAdjust,on=["ZIP_CODE","ZCTA"],how="outer")
del Data_2['HH_Ratio']

Data_2_HouseComp_No_Zip=Data_2_HouseComp[~Data_2_HouseComp['ZCTA'].isin(Data_2['ZCTA'])]
Data_2_HouseComp_No_Zip['ZIP_CODE']=np.nan
Data_2_HouseComp_No_Zip=Data_2_HouseComp_No_Zip[Data_2.columns.tolist()]
Data_2=Data_2.append(Data_2_HouseComp_No_Zip)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [19]:
Data_3_Eco=pd.read_csv("/home/jian/Docs/Household_and_Population/2016/ACS_16_5YR_DP03_with_ann (Demo work force).csv",skiprows=1)

Data_3_Eco=Data_3_Eco.rename(columns={"Id2":"ZCTA"})

Data_3_Eco_Col=Data_3_Eco.columns.tolist()
Data_3_Eco_Col=[x for x in Data_3_Eco_Col if "Estimate" in x]
Data_3_NoAdjust_Col=[x for x in Data_3_Eco_Col if "Rate" in x]+[x for x in Data_3_Eco_Col if "ncome" in x]

Data_3_NoAdjust=Data_3_Eco[['ZCTA']+Data_3_NoAdjust_Col]
Data_3_NoAdjust=pd.merge(result_NoAjust,Data_3_NoAdjust,on=['ZCTA'],how='left')
Data_3=Data_3_NoAdjust.copy()

Data_3_Eco_No_Zip=Data_3_Eco[~Data_3_Eco['ZCTA'].isin(Data_3['ZCTA'])]
Data_3_Eco_No_Zip['ZIP_CODE']=np.nan
Data_3_Eco_No_Zip=Data_3_Eco_No_Zip[Data_3.columns.tolist()]
Data_3=Data_3.append(Data_3_Eco_No_Zip)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [20]:
Data_4_AgeGender=pd.read_csv("/home/jian/Docs/Household_and_Population/2016/ACS_16_5YR_B01001_with_ann (Age by Sex).csv",skiprows=1)

Data_4_AgeGender=Data_4_AgeGender.rename(columns={"Id2":"ZCTA"})

Data_4_AgeGender_Col=Data_4_AgeGender.columns.tolist()
Data_4_AgeGender_Col=[x for x in Data_4_AgeGender_Col if "Estimate" in x]

Data_4_PopAdjust=Data_4_AgeGender[['ZCTA']+Data_4_AgeGender_Col]
Data_4_PopAdjust=pd.merge(result_Pop,Data_4_PopAdjust,on=['ZCTA'],how='left')
for col in Data_4_AgeGender_Col:
    Data_4_PopAdjust[col]=Data_4_PopAdjust[col]*Data_4_PopAdjust['Pop_Ratio']
Data_4_PopAdjust.head(2)
del Data_4_PopAdjust['Pop_Ratio']
Data_4=Data_4_PopAdjust.copy()

Data_4_AgeGender_No_Zip=Data_4_AgeGender[~Data_4_AgeGender['ZCTA'].isin(Data_4['ZCTA'])]
Data_4_AgeGender_No_Zip['ZIP_CODE']=np.nan
Data_4_AgeGender_No_Zip=Data_4_AgeGender_No_Zip[Data_4.columns.tolist()]
Data_4=Data_4.append(Data_4_AgeGender_No_Zip)

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (4,6,8,14,16,24,26,32,34,54,56,62,64,72,74,80,82) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [21]:
result=pd.merge(result,Data_1,on=['ZIP_CODE','ZCTA'],how="left")
result=pd.merge(result,Data_2,on=['ZIP_CODE','ZCTA'],how="left")
result=pd.merge(result,Data_3,on=['ZIP_CODE','ZCTA'],how="left")
result=pd.merge(result,Data_4,on=['ZIP_CODE','ZCTA'],how="left")
result.to_csv("/home/jian/Docs/Household_and_Population/2016/Demo_Dataset_2018EASI.csv",index=False)